In [3]:
# Active env should say "nbai"
!conda info | grep 'active env'

     active environment : nbai
    active env location : C:\Users\percy\anaconda3\envs\nbai


In [1]:
import torch
from torch.nn import functional as F
from torch import nn
from src.graph_format import NbaDatabase, NbaPosessionDatabase

In [8]:
database = NbaPosessionDatabase([2016])
#database.flush()
#database.populate()

print(len(database))

252646


In [9]:

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)

torch.manual_seed(42)
train_size = int(len(database) * 0.9)
val_size = len(database) - train_size
train_set, val_set = torch.utils.data.random_split(database, [train_size, val_size])
train_loader = torch.utils.data.DataLoader(train_set, batch_size=640, shuffle=True, pin_memory=use_cuda)
val_loader = torch.utils.data.DataLoader(val_set, batch_size=640, shuffle=True, pin_memory=use_cuda)

cpu


In [10]:
class pownet(nn.Module):
    def __init__(self):
        super(pownet, self).__init__()
        self.bn = nn.BatchNorm1d(6)
        self.order_1 = nn.Linear(3, 1, bias=True)
        self.order_2 = nn.Linear(3, 1, bias=True)

        self.order_3 = nn.Linear(3, 1, bias=True)

    def forward(self, x):
        x = self.bn(x)
        score_diff = x[:, 0] - x[:, 1]
        penalty_diff = x[:, 2] - x[:, 3]
        time_left = x[:, 4:5]
        has_pos = x[:, 5]
        state_vec = torch.stack((score_diff, penalty_diff, has_pos), dim=1)
        order_1 = self.order_1(state_vec)
        order_2 = self.order_2(state_vec) / time_left
        #order_3 = self.order_3(state_vec) / (time_left ** 2)
        out = torch.sigmoid(order_1 + order_2) #+ order_3)
        return out


In [11]:
criterion = nn.BCELoss()
net = pownet()
net = net.to(device)
optimizer = torch.optim.SGD(net.parameters(), lr=1e-2, momentum=0.9)
scheduler = torch.optim.lr_scheduler.CyclicLR(optimizer, base_lr=1e-4, max_lr=1e-2, step_size_up=50)

In [12]:
running_loss = 0
for epoch in range(100):
    for i, (start_state, end_state, label) in enumerate(train_loader):
        start_prob = net(start_state.to(device))
        end_prob = net(end_state.to(device))
        loss = criterion(start_prob, label.to(device)) + criterion(end_prob, label.to(device))
        running_loss += loss.item()
        if i % 10000 == 0:
            print(epoch, "loss", running_loss, start_state[0].long().tolist(), "-", label[0].item(), "", start_prob[0].item())
            running_loss = 0
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        scheduler.step()


0 loss 3.3098220825195312 [53, 51, 0, 1, 1192, 1] - 0.0  0.6227660179138184


KeyboardInterrupt: 

In [7]:
print(len(database))

4584520


In [8]:
all_stats = Stats()
i = 0
for data in database:
    if i % 5000 == 0:
        print(i)
        print(all_stats)
    i += 1
    tensor = data[0]
    play = Play(tensor)
    all_stats += Stats(play)
print(all_stats)

NameError: name 'Stats' is not defined

In [ ]:
print(all_stats)

In [7]:

then = datetime.now()
for i in range(100000):
    s.get(i % 227, r)
datetime.now() - then

datetime.timedelta(seconds=5, microseconds=391649)